# H2K to HPXML Mapping

In [1]:
import xmlschema
import pandas as pd
from pprint import pprint


In [2]:
# Load the H2K xsd into 
h2k_schema = xmlschema.XMLSchema('./schemas/h2k/H2k Schema.xsd')

First I want to make sure the schema is still valid. I used a H2K file from the HTAP new archetypes directory as a sample.

In [3]:
h2k_schema.is_valid('./data/h2k/ERS-1032.H2K')

True

In [4]:
# Convert xml to dictionary
pprint(h2k_schema.to_dict('./data/h2k/ERS-1032.H2K'))

{'@uiUnits': 'Imperial',
 '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema',
 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@{http://www.w3.org/XML/1998/namespace}lang': 'en',
 'AllResults': {'Results': [{'@houseCode': 'UserHouse',
                             '@sha256': 'D84D9C13BA8A4F9354B457881AC812CB5D22AADB207E4B2FD8457A49C99AE98D',
                             'Annual': {'ActualFuelCosts': {'@electrical': Decimal('1238.419799805'),
                                                            '@naturalGas': Decimal('0E-9'),
                                                            '@oil': Decimal('0E-9'),
                                                            '@propane': Decimal('0E-9'),
                                                            '@wood': Decimal('0E-9')},
                                        'AirChangeRate': {'@natural': Decimal('0.026878394'),
                                                          '@total': Decimal('0.075582661')},
    

                                       'GrossArea': {'@buildingSurfaceArea': Decimal('316.661773682'),
                                                     '@ceiling': Decimal('89.930099487'),
                                                     '@doors': Decimal('3.485154390'),
                                                     '@exposedFloors': Decimal('23.968997955'),
                                                     '@houseVolumeWithoutCrawlspace': Decimal('511.883728027'),
                                                     '@ponyWall': Decimal('0E-9'),
                                                     '@slab': Decimal('0E-9'),
                                                     'Basement': {'@aboveGrade': Decimal('6.038698196'),
                                                                  '@belowGrade': Decimal('42.270885468'),
                                                                  '@floorHeader': Decimal('4.552248955'),
                                 

                                                                             '@february': Decimal('0.00000'),
                                                                             '@january': Decimal('0.00000'),
                                                                             '@july': Decimal('0.00000'),
                                                                             '@june': Decimal('0.00000'),
                                                                             '@march': Decimal('0.00000'),
                                                                             '@may': Decimal('0.00000'),
                                                                             '@november': Decimal('0.00000'),
                                                                             '@october': Decimal('0.00000'),
                                                                             '@september': Decimal('0.00000')},
                          

                                                     'MainFloors': {'@mainWalls': Decimal('91.929382324'),
                                                                    'Door': {'@grossArea': Decimal('3.996121407'),
                                                                             '@rsiValue': Decimal('1.139625430')},
                                                                    'Windows': {'East': {'@grossArea': Decimal('0E-9'),
                                                                                         '@rsiValue': Decimal('0E-9')},
                                                                                'North': {'@grossArea': Decimal('4.598701000'),
                                                                                          '@rsiValue': Decimal('0.800829470')},
                                                                                'NorthEast': {'@grossArea': Decimal('0E-9'),
                                          

                                                                                            'Type': {'@code': '7',
                                                                                                     'English': 'Waferboard/Oriented '
                                                                                                                'strand '
                                                                                                                'board',
                                                                                                     'French': 'Panneau '
                                                                                                               'de '
                                                                                                               'copeaux/de '
                                                                                                               'copeaux '
                     

### Building Components
H2K uses a nested architecture to store building components data. I just loaded a subset of H2K into a dictionary in order to evaluate feasibility of extracting building measurements.

In [5]:
component_dict = h2k_schema.to_dict('./data/h2k/ERS-1032.H2K', 'House/Components')

In [6]:
component_dict.keys()

dict_keys(['Ceiling', 'Wall', 'Floor', 'Basement', 'HotWater'])

### Building Measurements

In [7]:
# Extract wall measurements

wall_height = []
wall_perimeter = []

for walls in component_dict['Wall']:
    wall_height.append(float(walls['Measurements']['@height']))
    wall_perimeter.append(float(walls['Measurements']['@perimeter']))

In [8]:
print('wall heights: {}'.format(wall_height))
print('wall perimeters: {}'.format(wall_perimeter))

wall heights: [2.7737, 2.7737, 2.4384]
wall perimeters: [9.9974, 9.9365, 13.4112]


In [9]:
def find_key(d, value):
    for k,v in d.items():
        if k == value:
            return [k,v]
        elif type(v) is list:
            for i in range(len(v)):
                p = find_key(v[i], value)
                if p:
                    return [k] + p
        elif type(v) is dict:
            p = find_key(v, value)
            if p:
                return [k] + p

In [10]:
for key, component in component_dict.items():
    for i in range(len(component)):
        measurements = find_key(component[i], 'Measurements')
        if measurements:
            print(key, measurements[-1])

Ceiling {'@area': Decimal('89.9301'), '@heelHeight': Decimal('0.4572'), '@length': Decimal('13.716'), 'Slope': {'@code': '4', '@value': Decimal('0.333'), 'English': '4 / 12', 'French': '4 / 12'}}
Wall {'@height': Decimal('2.7737'), '@perimeter': Decimal('9.9974')}
Wall {'@height': Decimal('2.7737'), '@perimeter': Decimal('9.9365')}
Wall {'@height': Decimal('2.4384'), '@perimeter': Decimal('13.4112')}
Floor {'@area': Decimal('23.969'), '@length': Decimal('9.9974')}
Basement {'@area': Decimal('89.9301'), '@isRectangular': False, '@perimeter': Decimal('39.624')}
